## Style Transfer Embeddings Code

In [ ]:
'''
code adapted from: https://www.tensorflow.org/tutorials/generative/style_transfer
'''
import tensorflow as tf

import IPython.display as display

import matplotlib.pyplot as plt
import matplotlib as mpl
mpl.rcParams['figure.figsize'] = (12,12)
mpl.rcParams['axes.grid'] = False

import numpy as np
import PIL.Image
import time
import functools

import pathlib
import glob

import pickle
import os

In [2]:
def tensor_to_image(tensor):
    tensor = tensor*255
    tensor = np.array(tensor, dtype=np.uint8)
    if np.ndim(tensor)>3:
        assert tensor.shape[0] == 1
    tensor = tensor[0]
    return PIL.Image.fromarray(tensor)

def load_img(path_to_img):
    max_dim = 512
    img = tf.io.read_file(path_to_img)
    img = tf.image.decode_image(img, channels=3)
    img = tf.image.convert_image_dtype(img, tf.float32)

    shape = tf.cast(tf.shape(img)[:-1], tf.float32)
    long_dim = max(shape)
    scale = max_dim / long_dim

    new_shape = tf.cast(shape * scale, tf.int32)

    img = tf.image.resize(img, new_shape)
    img = img[tf.newaxis, :]
    return img


def imshow(image, title=None):
    if len(image.shape) > 3:
        image = tf.squeeze(image, axis=0)

    plt.imshow(image)
    if title:
        plt.title(title)


In [3]:
'''
x = tf.keras.applications.vgg19.preprocess_input(content_image*255)
x = tf.image.resize(x, (224, 224))
vgg = tf.keras.applications.VGG19(include_top=True, weights='imagenet')
prediction_probabilities = vgg(x)
prediction_probabilities.shape
'''

"\nx = tf.keras.applications.vgg19.preprocess_input(content_image*255)\nx = tf.image.resize(x, (224, 224))\nvgg = tf.keras.applications.VGG19(include_top=True, weights='imagenet')\nprediction_probabilities = vgg(x)\nprediction_probabilities.shape\n"

In [4]:
def vgg_layers(layer_names):
    """ Creates a vgg model that returns a list of intermediate output values."""
    # Load our model. Load pretrained VGG, trained on imagenet data
    weights_path = 'vgg19_weights_tf_dim_ordering_tf_kernels_notop.h5'
    vgg = tf.keras.applications.VGG19(include_top=False, weights=weights_path)
    vgg.trainable = False

    outputs = [vgg.get_layer(name).output for name in layer_names]

    model = tf.keras.Model([vgg.input], outputs)
    return model

def gram_matrix(input_tensor):
    result = tf.linalg.einsum('bijc,bijd->bcd', input_tensor, input_tensor)
    input_shape = tf.shape(input_tensor)
    num_locations = tf.cast(input_shape[1]*input_shape[2], tf.float32)
    return result/(num_locations)

class StyleContentModel(tf.keras.models.Model):
    def __init__(self, style_layers, content_layers):
        super(StyleContentModel, self).__init__()
        self.vgg =  vgg_layers(style_layers + content_layers)
        self.style_layers = style_layers
        self.content_layers = content_layers
        self.num_style_layers = len(style_layers)
        self.vgg.trainable = False

    def call(self, inputs):
        "Expects float input in [0,1]"
        inputs = inputs*255.0
        preprocessed_input = tf.keras.applications.vgg19.preprocess_input(inputs)
        outputs = self.vgg(preprocessed_input)
        style_outputs, content_outputs = (outputs[:self.num_style_layers],outputs[self.num_style_layers:])

        style_outputs = [gram_matrix(style_output) for style_output in style_outputs]

        content_dict = {content_name:value for content_name, value in zip(self.content_layers, content_outputs)}

        style_dict = {style_name:value for style_name, value in zip(self.style_layers, style_outputs)}

        return {'content':content_dict, 'style':style_dict}

In [7]:
data_dir = 'Stripe Images/Convolutional DNN/cluster0'
image_files = glob.glob(data_dir+'/*.png')
image_count = len(image_files)

In [8]:
image_count

1000

In [9]:
##Define the extractor
weights_path = 'vgg19_weights_tf_dim_ordering_tf_kernels_notop.h5'
vgg = tf.keras.applications.VGG19(include_top=False, weights=weights_path)
#for layer in vgg.layers:
#    print(layer.name)

In [10]:
print("loaded weights")
content_layers = ['block5_conv2']
style_layers = ['block1_conv1',  
                'block2_conv1',
                'block3_conv1']
                #'block4_conv1',
                #'block5_conv1'

loaded weights


In [11]:

num_content_layers = len(content_layers)
num_style_layers = len(style_layers)
extractor = StyleContentModel(style_layers, content_layers)


In [26]:
image_style_embeddings = {}

In [27]:
i = 0

In [28]:
folder_dir = "Stripe Images/Convolutional DNN/"
for folder in os.listdir(folder_dir):
    if ".DS_" not in folder:
        for filename in os.listdir(folder_dir + folder):
            if filename.endswith(".png"):
                print('Processing image {}/{}'.format( i+1, image_count ))
                image_data = load_img(folder_dir + folder + "/" + filename)
                style = extractor(tf.constant(image_data))['style']
                vec = np.hstack([np.ravel(s) for s in style.values()])
                image_style_embeddings[filename] = vec
                i += 1


Processing image 1/1000
Processing image 2/1000
Processing image 3/1000
Processing image 4/1000
Processing image 5/1000
Processing image 6/1000
Processing image 7/1000
Processing image 8/1000
Processing image 9/1000
Processing image 10/1000
Processing image 11/1000
Processing image 12/1000
Processing image 13/1000
Processing image 14/1000
Processing image 15/1000
Processing image 16/1000
Processing image 17/1000
Processing image 18/1000
Processing image 19/1000
Processing image 20/1000
Processing image 21/1000
Processing image 22/1000
Processing image 23/1000
Processing image 24/1000
Processing image 25/1000
Processing image 26/1000
Processing image 27/1000
Processing image 28/1000
Processing image 29/1000
Processing image 30/1000
Processing image 31/1000
Processing image 32/1000
Processing image 33/1000
Processing image 34/1000
Processing image 35/1000
Processing image 36/1000
Processing image 37/1000
Processing image 38/1000
Processing image 39/1000
Processing image 40/1000
Processin

Processing image 321/1000
Processing image 322/1000
Processing image 323/1000
Processing image 324/1000
Processing image 325/1000
Processing image 326/1000
Processing image 327/1000
Processing image 328/1000
Processing image 329/1000
Processing image 330/1000
Processing image 331/1000
Processing image 332/1000
Processing image 333/1000
Processing image 334/1000
Processing image 335/1000
Processing image 336/1000
Processing image 337/1000
Processing image 338/1000
Processing image 339/1000
Processing image 340/1000
Processing image 341/1000
Processing image 342/1000
Processing image 343/1000
Processing image 344/1000
Processing image 345/1000
Processing image 346/1000
Processing image 347/1000
Processing image 348/1000
Processing image 349/1000
Processing image 350/1000
Processing image 351/1000
Processing image 352/1000
Processing image 353/1000
Processing image 354/1000
Processing image 355/1000
Processing image 356/1000
Processing image 357/1000
Processing image 358/1000
Processing i

Processing image 637/1000
Processing image 638/1000
Processing image 639/1000
Processing image 640/1000
Processing image 641/1000
Processing image 642/1000
Processing image 643/1000
Processing image 644/1000
Processing image 645/1000
Processing image 646/1000
Processing image 647/1000
Processing image 648/1000
Processing image 649/1000
Processing image 650/1000
Processing image 651/1000
Processing image 652/1000
Processing image 653/1000
Processing image 654/1000
Processing image 655/1000
Processing image 656/1000
Processing image 657/1000
Processing image 658/1000
Processing image 659/1000
Processing image 660/1000
Processing image 661/1000
Processing image 662/1000
Processing image 663/1000
Processing image 664/1000
Processing image 665/1000
Processing image 666/1000
Processing image 667/1000
Processing image 668/1000
Processing image 669/1000
Processing image 670/1000
Processing image 671/1000
Processing image 672/1000
Processing image 673/1000
Processing image 674/1000
Processing i

Processing image 953/1000
Processing image 954/1000
Processing image 955/1000
Processing image 956/1000
Processing image 957/1000
Processing image 958/1000
Processing image 959/1000
Processing image 960/1000
Processing image 961/1000
Processing image 962/1000
Processing image 963/1000
Processing image 964/1000
Processing image 965/1000
Processing image 966/1000
Processing image 967/1000
Processing image 968/1000
Processing image 969/1000
Processing image 970/1000
Processing image 971/1000
Processing image 972/1000
Processing image 973/1000
Processing image 974/1000
Processing image 975/1000
Processing image 976/1000
Processing image 977/1000
Processing image 978/1000
Processing image 979/1000
Processing image 980/1000
Processing image 981/1000
Processing image 982/1000
Processing image 983/1000
Processing image 984/1000
Processing image 985/1000
Processing image 986/1000
Processing image 987/1000
Processing image 988/1000
Processing image 989/1000
Processing image 990/1000
Processing i

In [13]:
for i, image_file in enumerate(image_files):
    print('Processing image {}/{}'.format( i+1, image_count ))
    image_data = load_img(image_file)
    style = extractor(tf.constant(image_data))['style']
    vec = np.hstack([np.ravel(s) for s in style.values()])
    image_style_embeddings[image_file] = vec


Processing image 1/1000
Processing image 2/1000
Processing image 3/1000
Processing image 4/1000
Processing image 5/1000
Processing image 6/1000
Processing image 7/1000
Processing image 8/1000
Processing image 9/1000
Processing image 10/1000
Processing image 11/1000
Processing image 12/1000
Processing image 13/1000
Processing image 14/1000
Processing image 15/1000
Processing image 16/1000
Processing image 17/1000
Processing image 18/1000
Processing image 19/1000
Processing image 20/1000
Processing image 21/1000
Processing image 22/1000
Processing image 23/1000
Processing image 24/1000
Processing image 25/1000
Processing image 26/1000
Processing image 27/1000
Processing image 28/1000
Processing image 29/1000
Processing image 30/1000
Processing image 31/1000
Processing image 32/1000
Processing image 33/1000
Processing image 34/1000
Processing image 35/1000
Processing image 36/1000
Processing image 37/1000
Processing image 38/1000
Processing image 39/1000
Processing image 40/1000
Processin

Processing image 321/1000
Processing image 322/1000
Processing image 323/1000
Processing image 324/1000
Processing image 325/1000
Processing image 326/1000
Processing image 327/1000
Processing image 328/1000
Processing image 329/1000
Processing image 330/1000
Processing image 331/1000
Processing image 332/1000
Processing image 333/1000
Processing image 334/1000
Processing image 335/1000
Processing image 336/1000
Processing image 337/1000
Processing image 338/1000
Processing image 339/1000
Processing image 340/1000
Processing image 341/1000
Processing image 342/1000
Processing image 343/1000
Processing image 344/1000
Processing image 345/1000
Processing image 346/1000
Processing image 347/1000
Processing image 348/1000
Processing image 349/1000
Processing image 350/1000
Processing image 351/1000
Processing image 352/1000
Processing image 353/1000
Processing image 354/1000
Processing image 355/1000
Processing image 356/1000
Processing image 357/1000
Processing image 358/1000
Processing i

Processing image 637/1000
Processing image 638/1000
Processing image 639/1000
Processing image 640/1000
Processing image 641/1000
Processing image 642/1000
Processing image 643/1000
Processing image 644/1000
Processing image 645/1000
Processing image 646/1000
Processing image 647/1000
Processing image 648/1000
Processing image 649/1000
Processing image 650/1000
Processing image 651/1000
Processing image 652/1000
Processing image 653/1000
Processing image 654/1000
Processing image 655/1000
Processing image 656/1000
Processing image 657/1000
Processing image 658/1000
Processing image 659/1000
Processing image 660/1000
Processing image 661/1000
Processing image 662/1000
Processing image 663/1000
Processing image 664/1000
Processing image 665/1000
Processing image 666/1000
Processing image 667/1000
Processing image 668/1000
Processing image 669/1000
Processing image 670/1000
Processing image 671/1000
Processing image 672/1000
Processing image 673/1000
Processing image 674/1000
Processing i

Processing image 953/1000
Processing image 954/1000
Processing image 955/1000
Processing image 956/1000
Processing image 957/1000
Processing image 958/1000
Processing image 959/1000
Processing image 960/1000
Processing image 961/1000
Processing image 962/1000
Processing image 963/1000
Processing image 964/1000
Processing image 965/1000
Processing image 966/1000
Processing image 967/1000
Processing image 968/1000
Processing image 969/1000
Processing image 970/1000
Processing image 971/1000
Processing image 972/1000
Processing image 973/1000
Processing image 974/1000
Processing image 975/1000
Processing image 976/1000
Processing image 977/1000
Processing image 978/1000
Processing image 979/1000
Processing image 980/1000
Processing image 981/1000
Processing image 982/1000
Processing image 983/1000
Processing image 984/1000
Processing image 985/1000
Processing image 986/1000
Processing image 987/1000
Processing image 988/1000
Processing image 989/1000
Processing image 990/1000
Processing i

In [29]:
with open('stripe_style_vectors','wb') as f:
    pickle.dump(image_style_embeddings, f)

In [25]:
len(image_style_embeddings)

1